In [417]:
library(tidyverse)
library(dplyr)
library(reshape2)

In [418]:
ortholog_info = read.csv("ortholog_ID.csv")
head(ortholog_info)


,GeneID,Gene
,<int>,<chr>
1,1,S.bayanusvar.uvarum (Suva_16.102)
2,2,K.africana (KAFR0K00320)
3,3,T.blattae (TBLA0A02040)
4,4,S.kudriavzevii (Skud_16.75)
5,5,K.africana (KAFR0C02290)
6,6,T.blattae (TBLA0B05310)


In [419]:
screening_info = read.csv("Dox_plate_info_20230830.csv")
head(screening_info)

,Printing_order,Medium,Plate,Plasmid_qA,Plasmid_qB,Plasmid_qC,Plasmid_qD,IMG_S2,Dox1_batch,Image,Dox3_batch
,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>,<int>,<chr>
1,1,YPD,1,25,1,2,26,IMG_00049,d000264,1,Sel_3
2,2,YPD,2,25,3,4,26,IMG_00050,d000264,2,Sel_3
3,3,YPD,3,25,5,6,26,IMG_00051,d000264,3,Sel_3
4,4,YPD,4,25,7,8,26,IMG_00052,d000264,4,Sel_3
5,5,YPD,5,25,9,10,26,IMG_00053,d000264,5,Sel_3
6,6,YPD,6,25,11,12,26,IMG_00054,d000264,6,Sel_3


In [420]:
screening_info_table = data.frame(A = screening_info$Plasmid_qA,
                                  B = screening_info$Plasmid_qB,
                                  C = screening_info$Plasmid_qC,
                                  D = screening_info$Plasmid_qD,
                                  Plate = screening_info$Image,
                                  Medium = screening_info$Medium,
                                  Sample_Plate = screening_info$Plate
                                 )

In [421]:
screening_info_table = melt(screening_info_table,id.vars=c("Sample_Plate","Plate","Medium"))

In [422]:
screening_info_table$PlateQuadrant = paste(screening_info_table$Plate,screening_info_table$variable,sep=".")
screening_info_table$GeneID = screening_info_table$value
screening_info_table = merge(screening_info_table,ortholog_info , by = 'GeneID')
head(screening_info_table)

,GeneID,Sample_Plate,Plate,Medium,variable,value,PlateQuadrant,Gene
,<int>,<int>,<int>,<chr>,<fct>,<int>,<chr>,<chr>
1,1,1,1,YPD,B,1,1.B,S.bayanusvar.uvarum (Suva_16.102)
2,1,1,14,Dox,B,1,14.B,S.bayanusvar.uvarum (Suva_16.102)
3,2,1,1,YPD,C,2,1.C,K.africana (KAFR0K00320)
4,2,1,14,Dox,C,2,14.C,K.africana (KAFR0K00320)
5,3,2,15,Dox,B,3,15.B,T.blattae (TBLA0A02040)
6,3,2,2,YPD,B,3,2.B,T.blattae (TBLA0A02040)


In [423]:
quadrant_gene = data.frame(ImageQuadrant = screening_info_table$PlateQuadrant,
                           Gene = screening_info_table$Gene)

In [424]:
pos_info_384 = read.csv("Plate_384_dox_coordinates.csv")
# This 384 pos is printed on each plate, and quadarent A-D is as in Dox_plate_info
pos_info_384$Pos384 = paste(paste("R",pos_info_384$r384,sep=""),paste("C",pos_info_384$c384,sep=""),sep="-")
#head(pos_info_384)
pos_info_384_border_remove = pos_info_384[pos_info_384$Gene != "Border",]
pos_info_384_border_remove$GeneID = pos_info_384_border_remove$Gene
pos_info_384_border_remove$Gene = NULL

In [425]:
pos_info_384_border_remove = merge(pos_info_384_border_remove,ortholog_info , by = 'GeneID')
head(pos_info_384_border_remove)

,GeneID,c384,r384,Pos384,Gene
,<chr>,<int>,<int>,<chr>,<chr>
1,1,14,5,R5-C14,S.bayanusvar.uvarum (Suva_16.102)
2,1,2,9,R9-C2,S.bayanusvar.uvarum (Suva_16.102)
3,1,4,15,R15-C4,S.bayanusvar.uvarum (Suva_16.102)
4,1,4,9,R9-C4,S.bayanusvar.uvarum (Suva_16.102)
5,1,10,5,R5-C10,S.bayanusvar.uvarum (Suva_16.102)
6,1,18,9,R9-C18,S.bayanusvar.uvarum (Suva_16.102)


In [426]:
array_gene =  data.frame(Pos384    = pos_info_384_border_remove$Pos384,
                         ArrayGene = pos_info_384_border_remove$Gene)

In [427]:
image_data = read.csv("2022-11-08_Dox_AUC_data.csv")
head(image_data)

,row,column,spot_well,Plate_ID,Selection,auc
,<int>,<int>,<chr>,<int>,<chr>,<dbl>
1,1,21,A21,10,Sel_3,62262.0
2,1,10,A10,10,Sel_3,52594.0
3,1,7,A7,10,Sel_3,54963.0
4,1,9,A9,10,Sel_3,50046.5
5,1,12,A12,10,Sel_3,51087.0
6,1,6,A6,10,Sel_3,52430.5


In [428]:
df = data.frame("Plate"  =image_data$Plate_ID,
                "Row1536"=image_data$row,
                "Col1536"=image_data$column,
                "Selection"=image_data$Selection,
                "AUC"    =image_data$auc
                
               )

In [429]:
df2 = reshape(df, idvar = c("Plate","Row1536","Col1536"), timevar = "Selection", direction = "wide")

In [430]:
#df$PlateID = as.numeric(df$PlateID)
head(df2)

,Plate,Row1536,Col1536,AUC.Sel_3,AUC.Sel_2
,<int>,<int>,<int>,<dbl>,<dbl>
1,10,1,21,62262.0,58299.5
2,10,1,10,52594.0,58811.5
3,10,1,7,54963.0,57452.5
4,10,1,9,50046.5,56546.5
5,10,1,12,51087.0,58333.5
6,10,1,6,52430.5,55763.0


In [431]:
df2$Row384 =  ceiling(df2$Row1536/2) 
df2$Col384 =  ceiling(df2$Col1536/2)
df2$Pos384 =  paste(paste("R",df2$Row384,sep=""),paste("C",df2$Col384,sep=""),sep="-")

medium_df = screening_info_table %>% select(Plate,Medium,Sample_Plate)
df2 = merge(df2,medium_df,by="Plate") %>% distinct()

head(df2)

,Plate,Row1536,Col1536,AUC.Sel_3,AUC.Sel_2,Row384,Col384,Pos384,Medium,Sample_Plate
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<int>
1,1,1,17,54047.5,50409.5,1,9,R1-C9,YPD,1
2,1,1,11,53493.0,48631.0,1,6,R1-C6,YPD,1
3,1,1,15,58773.0,51594.5,1,8,R1-C8,YPD,1
4,1,1,13,55655.5,50385.5,1,7,R1-C7,YPD,1
5,1,1,23,59384.0,52376.0,1,12,R1-C12,YPD,1
6,1,1,46,62104.5,55314.0,1,23,R1-C23,YPD,1


In [432]:
unique(df2$Medium)
df2 %>% filter(Medium=="Dox")

[1] "YPD" "Dox"

Plate,Row1536,Col1536,AUC.Sel_3,AUC.Sel_2,Row384,Col384,Pos384,Medium,Sample_Plate
<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<int>
14,1,15,51970.0,55573.0,1,8,R1-C8,Dox,1
14,1,9,52504.5,54999.5,1,5,R1-C5,Dox,1
14,1,21,52310.5,54783.5,1,11,R1-C11,Dox,1
14,1,30,54742.0,53766.5,1,15,R1-C15,Dox,1
14,1,7,62644.0,52749.0,1,4,R1-C4,Dox,1
14,1,18,51220.5,45839.0,1,9,R1-C9,Dox,1
14,1,22,54390.0,50309.5,1,11,R1-C11,Dox,1
14,1,43,55571.5,58975.0,1,22,R1-C22,Dox,1
14,1,10,52029.0,58832.0,1,5,R1-C5,Dox,1


In [433]:
df_query_gene_pos = merge(df2,array_gene , by = 'Pos384')
head(df_query_gene_pos)

,Pos384,Plate,Row1536,Col1536,AUC.Sel_3,AUC.Sel_2,Row384,Col384,Medium,Sample_Plate,ArrayGene
,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<chr>
1,R10-C10,6,19,19,50896.0,53016.5,10,10,YPD,6,N.dairenensis (NDAI0I01070)
2,R10-C10,5,20,20,58168.0,48312.0,10,10,YPD,5,N.dairenensis (NDAI0I01070)
3,R10-C10,14,19,19,48659.5,46260.0,10,10,Dox,1,N.dairenensis (NDAI0I01070)
4,R10-C10,8,20,19,47996.0,44541.0,10,10,YPD,8,N.dairenensis (NDAI0I01070)
5,R10-C10,14,19,20,49235.5,44361.5,10,10,Dox,1,N.dairenensis (NDAI0I01070)
6,R10-C10,19,20,19,46188.5,44224.0,10,10,Dox,6,N.dairenensis (NDAI0I01070)


In [434]:
quadrants = c("1.1" = "A",
              "1.0" = "B",
              "0.1" = "C",
              "0.0" = "D"
              )
df_query_gene_pos$Quadrant = quadrants[paste(df_query_gene_pos$Row1536%%2,df_query_gene_pos$Col1536%%2,sep=".")]
df_query_gene_pos$PlateQuadrant = paste(df_query_gene_pos$Plate,df_query_gene_pos$Quadrant,sep=".")


In [435]:
head(df_query_gene_pos)
unique(df_query_gene_pos$Plate)


,Pos384,Plate,Row1536,Col1536,AUC.Sel_3,AUC.Sel_2,Row384,Col384,Medium,Sample_Plate,ArrayGene,Quadrant,PlateQuadrant
,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>
1,R10-C10,6,19,19,50896.0,53016.5,10,10,YPD,6,N.dairenensis (NDAI0I01070),A,6.A
2,R10-C10,5,20,20,58168.0,48312.0,10,10,YPD,5,N.dairenensis (NDAI0I01070),D,5.D
3,R10-C10,14,19,19,48659.5,46260.0,10,10,Dox,1,N.dairenensis (NDAI0I01070),A,14.A
4,R10-C10,8,20,19,47996.0,44541.0,10,10,YPD,8,N.dairenensis (NDAI0I01070),C,8.C
5,R10-C10,14,19,20,49235.5,44361.5,10,10,Dox,1,N.dairenensis (NDAI0I01070),B,14.B
6,R10-C10,19,20,19,46188.5,44224.0,10,10,Dox,6,N.dairenensis (NDAI0I01070),C,19.C


[1]  6  5 14  8 19 17  9 11 16 22 23 18 15  1  4  7 25 20 26 12 13 10 24  2 21
[26]  3

In [436]:
screening_info_table$Plate = NULL
screening_info_table$Medium = NULL
screening_info_table$Sample_Plate = NULL

head(screening_info_table)
head(df_query_gene_pos)

,GeneID,variable,value,PlateQuadrant,Gene
,<int>,<fct>,<int>,<chr>,<chr>
1,1,B,1,1.B,S.bayanusvar.uvarum (Suva_16.102)
2,1,B,1,14.B,S.bayanusvar.uvarum (Suva_16.102)
3,2,C,2,1.C,K.africana (KAFR0K00320)
4,2,C,2,14.C,K.africana (KAFR0K00320)
5,3,B,3,15.B,T.blattae (TBLA0A02040)
6,3,B,3,2.B,T.blattae (TBLA0A02040)


,Pos384,Plate,Row1536,Col1536,AUC.Sel_3,AUC.Sel_2,Row384,Col384,Medium,Sample_Plate,ArrayGene,Quadrant,PlateQuadrant
,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>
1,R10-C10,6,19,19,50896.0,53016.5,10,10,YPD,6,N.dairenensis (NDAI0I01070),A,6.A
2,R10-C10,5,20,20,58168.0,48312.0,10,10,YPD,5,N.dairenensis (NDAI0I01070),D,5.D
3,R10-C10,14,19,19,48659.5,46260.0,10,10,Dox,1,N.dairenensis (NDAI0I01070),A,14.A
4,R10-C10,8,20,19,47996.0,44541.0,10,10,YPD,8,N.dairenensis (NDAI0I01070),C,8.C
5,R10-C10,14,19,20,49235.5,44361.5,10,10,Dox,1,N.dairenensis (NDAI0I01070),B,14.B
6,R10-C10,19,20,19,46188.5,44224.0,10,10,Dox,6,N.dairenensis (NDAI0I01070),C,19.C


In [437]:
df_query_gene_pos_quadarant = merge(y=df_query_gene_pos,x=screening_info_table , by = c('PlateQuadrant'))

In [438]:
head(df_query_gene_pos_quadarant)

,PlateQuadrant,GeneID,variable,value,Gene,Pos384,Plate,Row1536,Col1536,AUC.Sel_3,AUC.Sel_2,Row384,Col384,Medium,Sample_Plate,ArrayGene,Quadrant
,<chr>,<int>,<fct>,<int>,<chr>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<chr>,<chr>
1,1.A,25,A,25,S.cerevisiae (YPL204W),R4-C22,1,7,43,62572.0,55285.5,4,22,YPD,1,S.kudriavzevii (Skud_16.75),A
2,1.A,25,A,25,S.cerevisiae (YPL204W),R7-C8,1,13,15,57872.5,51492.5,7,8,YPD,1,T.phaffii (TPHA0J02370),A
3,1.A,25,A,25,S.cerevisiae (YPL204W),R8-C11,1,15,21,60397.0,53409.5,8,11,YPD,1,H. sapiens (CSNK1D),A
4,1.A,25,A,25,S.cerevisiae (YPL204W),R10-C9,1,19,17,59950.0,52941.0,10,9,YPD,1,H. sapiens (CSNK1E),A
5,1.A,25,A,25,S.cerevisiae (YPL204W),R8-C10,1,15,19,59653.5,52851.5,8,10,YPD,1,S.mikatae (Smik_6.409),A
6,1.A,25,A,25,S.cerevisiae (YPL204W),R9-C11,1,17,21,61024.0,53397.0,9,11,YPD,1,T.delbrueckii (TDEL0E05230),A


In [439]:
unique(df_query_gene_pos_quadarant$Sample_Plate)
unique(df_query_gene_pos_quadarant$Plate)


[1]  1 10 11 12 13  2  3  4  5  6  7  8  9

[1]  1 10 11 12 13 14 15 16 17 18 19  2 20 21 22 23 24 25 26  3  4  5  6  7  8
[26]  9

In [440]:
unique(df_query_gene_pos_quadarant$Medium)
df_query_gene_pos_quadarant %>% filter(Medium=="Dox")

[1] "YPD" "Dox"

PlateQuadrant,GeneID,variable,value,Gene,Pos384,Plate,Row1536,Col1536,AUC.Sel_3,AUC.Sel_2,Row384,Col384,Medium,Sample_Plate,ArrayGene,Quadrant
<chr>,<int>,<fct>,<int>,<chr>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<chr>,<chr>
14.A,25,A,25,S.cerevisiae (YPL204W),R5-C19,14,9,37,52786.5,49635.5,5,19,Dox,1,T.delbrueckii (TDEL0E05230),A
14.A,25,A,25,S.cerevisiae (YPL204W),R13-C16,14,25,31,47253.5,45054.5,13,16,Dox,1,H. sapiens (CSNK1A1L),A
14.A,25,A,25,S.cerevisiae (YPL204W),R8-C14,14,15,27,46737.5,40478.0,8,14,Dox,1,N.castellii (NCAS0A01090),A
14.A,25,A,25,S.cerevisiae (YPL204W),R3-C21,14,5,41,51149.5,47146.5,3,21,Dox,1,C.glabrata (CAGL0H03553g),A
14.A,25,A,25,S.cerevisiae (YPL204W),R9-C2,14,17,3,45783.0,49817.5,9,2,Dox,1,S.bayanusvar.uvarum (Suva_16.102),A
14.A,25,A,25,S.cerevisiae (YPL204W),R8-C8,14,15,15,43629.5,41158.0,8,8,Dox,1,H. sapiens (CSNK1E),A
14.A,25,A,25,S.cerevisiae (YPL204W),R8-C5,14,15,9,42057.0,45735.0,8,5,Dox,1,H. sapiens (CSNK1A1L),A
14.A,25,A,25,S.cerevisiae (YPL204W),R15-C6,14,29,11,50408.5,52126.5,15,6,Dox,1,T.delbrueckii (TDEL0E05230),A
14.A,25,A,25,S.cerevisiae (YPL204W),R15-C16,14,29,31,47593.0,44518.0,15,16,Dox,1,H. sapiens (CSNK1A1L),A


In [441]:
data = data.frame(ArrayGene = df_query_gene_pos_quadarant$ArrayGene,
                  QueryGene = df_query_gene_pos_quadarant$Gene,
                  Plate     = df_query_gene_pos_quadarant$Plate,
                  Sample_Plate     = df_query_gene_pos_quadarant$Sample_Plate,
                  Row1536   = df_query_gene_pos_quadarant$Row1536,
                  Col1536   = df_query_gene_pos_quadarant$Col1536,
                  #PlateQuadrant = df_query_gene_pos_quadarant$PlateQuadrant,
                  Pos384    = df_query_gene_pos_quadarant$Pos384,
                  Medium    = df_query_gene_pos_quadarant$Medium,
                  #AUC_Sel2  = df_query_gene_pos_quadarant$AUC.Sel_2,
                  AUC_Sel3  = df_query_gene_pos_quadarant$AUC.Sel_3
                  
                 )
head(data)

,ArrayGene,QueryGene,Plate,Sample_Plate,Row1536,Col1536,Pos384,Medium,AUC_Sel3
,<chr>,<chr>,<int>,<int>,<int>,<int>,<chr>,<chr>,<dbl>
1,S.kudriavzevii (Skud_16.75),S.cerevisiae (YPL204W),1,1,7,43,R4-C22,YPD,62572.0
2,T.phaffii (TPHA0J02370),S.cerevisiae (YPL204W),1,1,13,15,R7-C8,YPD,57872.5
3,H. sapiens (CSNK1D),S.cerevisiae (YPL204W),1,1,15,21,R8-C11,YPD,60397.0
4,H. sapiens (CSNK1E),S.cerevisiae (YPL204W),1,1,19,17,R10-C9,YPD,59950.0
5,S.mikatae (Smik_6.409),S.cerevisiae (YPL204W),1,1,15,19,R8-C10,YPD,59653.5
6,T.delbrueckii (TDEL0E05230),S.cerevisiae (YPL204W),1,1,17,21,R9-C11,YPD,61024.0


In [442]:
#reshape to wide format for medium, and values of AUC
data_w <- dcast(data, ArrayGene + QueryGene+Sample_Plate+ Row1536+Col1536~ Medium,value.var="AUC_Sel3")
head(data_w)


#Divide Dox with YPD, to get Complementation score

,ArrayGene,QueryGene,Sample_Plate,Row1536,Col1536,Dox,YPD
,<chr>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>
1,C.glabrata (CAGL0H03553g),C.glabrata (CAGL0H03553g),5,4,11,42398.0,54593.5
2,C.glabrata (CAGL0H03553g),C.glabrata (CAGL0H03553g),5,6,41,47632.5,57008.0
3,C.glabrata (CAGL0H03553g),C.glabrata (CAGL0H03553g),5,10,7,43994.0,52270.5
4,C.glabrata (CAGL0H03553g),C.glabrata (CAGL0H03553g),5,10,35,49857.0,57969.0
5,C.glabrata (CAGL0H03553g),C.glabrata (CAGL0H03553g),5,14,35,52843.5,61392.0
6,C.glabrata (CAGL0H03553g),C.glabrata (CAGL0H03553g),5,18,27,46193.0,56509.0


In [443]:
data_w$CompScore = (data_w$Dox)/(data_w$YPD)
head(data_w)

,ArrayGene,QueryGene,Sample_Plate,Row1536,Col1536,Dox,YPD,CompScore
,<chr>,<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>
1,C.glabrata (CAGL0H03553g),C.glabrata (CAGL0H03553g),5,4,11,42398.0,54593.5,0.7766126
2,C.glabrata (CAGL0H03553g),C.glabrata (CAGL0H03553g),5,6,41,47632.5,57008.0,0.8355406
3,C.glabrata (CAGL0H03553g),C.glabrata (CAGL0H03553g),5,10,7,43994.0,52270.5,0.8416602
4,C.glabrata (CAGL0H03553g),C.glabrata (CAGL0H03553g),5,10,35,49857.0,57969.0,0.8600631
5,C.glabrata (CAGL0H03553g),C.glabrata (CAGL0H03553g),5,14,35,52843.5,61392.0,0.8607555
6,C.glabrata (CAGL0H03553g),C.glabrata (CAGL0H03553g),5,18,27,46193.0,56509.0,0.8174450


In [416]:
write.csv(data_w,"HRR25_orthologs_dox_screening_parsed_2023-08-30DEY.csv", row.names = FALSE)

In [445]:
data_w$pair = paste(data_w$ArrayGene, data_w$QueryGene,sep="-")

In [446]:
data_agg = data_w %>% group_by(pair) %>% 
summarize(mean = mean(CompScore),n = n(),SD=sd(CompScore) )#%>% 
#summarize(Sel3.mean = mean(AUC_Sel3),n_Sel3 = n())

In [447]:
head(data_agg)

pair,mean,n,SD
<chr>,<dbl>,<int>,<dbl>
C.glabrata (CAGL0H03553g)-C.glabrata (CAGL0H03553g),0.8241210,11,0.02914449
C.glabrata (CAGL0H03553g)-H. sapiens (CSNK1A1),0.8674966,11,0.03314069
C.glabrata (CAGL0H03553g)-H. sapiens (CSNK1A1L),0.8940396,11,0.04402071
C.glabrata (CAGL0H03553g)-H. sapiens (CSNK1D),0.9085575,11,0.05183176
C.glabrata (CAGL0H03553g)-H. sapiens (CSNK1E),0.8844831,11,0.03641463
C.glabrata (CAGL0H03553g)-K.africana (KAFR0C02290),0.9081177,11,0.02777594


In [449]:
data_agg2 = data_agg%>%
  separate(pair, c("ArrayGene", "QueryGene"), "-")
head(data_agg2)
write.csv(data_agg2,"HRR25_orthologs_dox_screening_parsed_agg_2023-08-30DEY.csv",row.names= FALSE)

ArrayGene,QueryGene,mean,n,SD
<chr>,<chr>,<dbl>,<int>,<dbl>
C.glabrata (CAGL0H03553g),C.glabrata (CAGL0H03553g),0.8241210,11,0.02914449
C.glabrata (CAGL0H03553g),H. sapiens (CSNK1A1),0.8674966,11,0.03314069
C.glabrata (CAGL0H03553g),H. sapiens (CSNK1A1L),0.8940396,11,0.04402071
C.glabrata (CAGL0H03553g),H. sapiens (CSNK1D),0.9085575,11,0.05183176
C.glabrata (CAGL0H03553g),H. sapiens (CSNK1E),0.8844831,11,0.03641463
C.glabrata (CAGL0H03553g),K.africana (KAFR0C02290),0.9081177,11,0.02777594
